# BugDetection Server
使用Flask框架搭建一个后端

### 1. 引入必要的包

In [ ]:
# -*- coding: utf-8 -*-    
#__author__="wowxinn" 

from flask import Flask, jsonify, abort
from flask import request    
from flask import make_response,Response    

import json   

import model_seq.predictor as predictor
from model_seq.predictor import SeqLSTM as lstm
import model_seq.vocabhandler as vocabhandler

### 2. 在这里定义一些函数与方法

In [ ]:
def Response_headers(content):    
    resp = Response(content)    
    resp.headers['Access-Control-Allow-Origin'] = '*'    
    return resp

### 3. 在这里存储一些服务器的数据

In [ ]:
tasks = [
    {
        'id': 1,
        'title': u'BugDetection',
        'description': u'wowxinn',
        'done': False
    }
]

predictor = lstm()
sess = predictor.sess
# predictor.predict([[9]])

### 4. 在这里定义了API的入口，使用了Flask的route机制

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return "Hello, World!"

@app.route('/apis/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = list(filter(lambda t: t['id'] == task_id, tasks)) ## 这里用list转了一下filter的返回类型
    if len(task) == 0:
        abort(404)
    return jsonify({'task': task[0]})

@app.route('/apis/predicts', methods=['POST'])  
def predicts():  
    if request.method == 'POST':    
        # POST:  
        # request.form获得所有post参数放在一个类似dict类中,to_dict()是字典化  
        # 单个参数可以通过request.form.to_dict().get("xxx","")获得  
        # ----------------------------------------------------  
        # GET:  
        # request.args获得所有get参数放在一个类似dict类中,to_dict()是字典化  
        # 单个参数可以通过request.args.to_dict().get('xxx',"")获得  
        
        inputs = request.get_json(force=True).get("input")
        print("[INFO]getting api sequence:",inputs["seq"])
        
        # 这里做了一个去掉unk API调用的过滤
        idx = [vocabhandler.get_id(w) for w in inputs["seq"] if vocabhandler.get_id(w) > 0] 
        print("[INFO]convert to id sequence:",idx)
        
        # 这里可以加一个预测，然后返回预测的内容
        (class_top_n_values, class_top_n_indices) = predictor.predict([idx])
        return jsonify({'predictValues': class_top_n_values[0].tolist(), 'predictIndices': class_top_n_indices[0].tolist()})
    else:    
        content = json.dumps({"error_code":"1001"})    
        resp = Response_headers(content)    
        return resp

### 5. 启动整个框架的入口

In [ ]:
if __name__ == '__main__':
    vocabhandler.load_vocab()
    app.run( 
        debug = True,
        use_reloader=False,
        port=5000
    )